
|  |
| ------------------------------------------------------- | 
| ![Tremplin des sciences](images/tremplinColorSmall.png) | 

Cahier d'exercices pour l'enseignement du changement climat climatique ou l'apprentissage de programmation issu de la collection "Climat et météo tremplin pour l'enseignement des sciences" (PIA IFÉ ENS de Lyon - Météofrance ENM Toulouse). Le dispositif clef en main repose sur l'utilisation d'une RaspberryPi chargée avec le système d'exploitation Debian enrichi, produit par le projet. Les sources et les exécutables sont accessibles dans [l'espace collaboratif du pojet à l'IFÉ ENS de Lyon](https://contrib-tremplin.ens-lyon.fr/) et une copie se trouve dans [l'espace collaboratif de la forge github](https://github.com/g-vidal/CahierDeProgrammes); plus d'information sur les [blogs d'accompagnement](http://blog.climatetmeteo.fr/GerardVidal/) systèmes d'exploitation sur [la page des OS  de Raspberries Pi](http://mediaserv.climatetmeteo.fr/images/RaspBerry/DebianStretchPi3/).  Toutes les ressources issues du projet sont fournies sous licence [Creative Commons](https://creativecommons.org/licenses/by-nc/4.0/) ou sous les licences libres d'origine des outils utilisés. Les ressources  du projet peuvent être utilisées dans tout autre environnement compatible. Les données en ligne sont accessibles sur le [serveur de données géolocalisées](http://geoloc-tremplin.ens-lyon.fr/climato-data/) `opendap` du projet tremplin.

![licence : Creative Commons](images/Licence.jpg) 

Auteur : G. Vidal

------------------------------------------------------------

# Une approche du changement climatique à Lyon : Création d'un jeu de données

Ce cahier d'exercices `ipython` propose une méthode d'exploration d'un jeu de données issues des simulations climatiques de Météofrance. Le lot utilisé  est issu de trois séries de modélisations  RCP 2.6 - 4.5 - 8.5. Le travail a été effectué avec la température maximale, et les données proposées contiennent la température minimale, la pluviométrie et l'humidité.  Le code pour la température maximale est directement transposable  aux autres variables du modèle. Cette première partie traite de la méhode pour qu'un enseignant puisse extraire les données dont il a besoin d'un fichier commandé sur le site [DRIAS](http://www.drias-climat.fr/), construire le fichier avec les données traitées puis le déposer sur le serveur pour le partager avvec les élèves. Ce cahier manipule des données multidimensionnelles et doit être réservé à des étudiants avancés si on souhaite l'utiliser en classe. Ce cahier utilise les données d'un carré 10 x 10 noeuds de la grille ALADIN centré sur la ville de Lyon. Le format d'entrée et de sortie est celui utilisé par les labos de climatologie, d'océanographie : `netCDF`.  Cet outil requiert l'installation des outils `netCDF4` et `numpy`

## Préparation de l'environnement et aperçu du fichier de données

Importer d'abord le module `netcdf4` et `numpy`, attention les majuscules sont impératives pour le nom `netCDF4`. Ces deux modules permettent de traiter  les fichiers multidimensionnels au format netCDF utilisés dans le monde de la météorologie et de l'océanographie principalement.

In [1]:
import netCDF4 as nc
import numpy as np
from datetime import datetime
from array import array
import sys, datetime, os

Importation des données de températures maximales depuis le fichier obtenu auprès du site [DRIAS](https://drias-prod.meteo.fr/okapi/accueil/okapiWebDrias/index.jsp) pour la région lyonnaise et intégration dans un fichier pour le traitement, puis affichage de la description du contenu, de la liste des variables.

L'exemple choisi ici a été réalisé avec une grille de 10 x 10 noeuds centrés sur la ville de Lyon, pour obtenir un jeu de données depuis le site [DRIAS](https://drias-prod.meteo.fr/okapi/accueil/okapiWebDrias/index.jsp) se reporter au manuel numérique réalisé par E. Le Jan et C. Larose dans le cadre du projet "Climat et Météo Tremplin pour l'enseignement des sciences". 

Le bloc de code ci-dessous est fourni à des fins de _"nettoyage et information"_ il n'est pas nécessaire de l'exécuter pour atteindre le résultat il permet d'obtenir les informations sur le fichier manipulé et enlève d'éventuelles scories d'exécutionq précédentes.

In [2]:
# Attention Avant de recharger les fichiers locaux depuis  les sources en ligne, 
# si des fichiers locaux sont OUVERTS
# il est nécessaire de les  fermer pour éviter des erreurs de données
# Dans tous les autres  situations
# L'exécution de ce bloc est inutile.
# Close Datasets if necessary
tMax_26_Lyon = nc.Dataset('http://geoloc-tremplin.ens-lyon.fr/climato-data/Lyon-1/tasmax_metro_CNRM_Aladin_rcp2.6_QT_RCP2.6_20060101-21001231.nc',casting='safe')
print('Descripteur des données : \n\n',tMax_26_Lyon)
try: os.remove('tMax_26_Lyon')  # par sécurité efface le fichier portant ce nom ! attention aux pertes possibles
except OSError : pass
try: os.remove('tMax_45_Lyon')  # par sécurité efface le fichier portant ce nom ! attention aux pertes possibles
except OSError : pass
try: os.remove('tMax_85_Lyon')  # par sécurité efface le fichier portant ce nom ! attention aux pertes possibles
except OSError : pass


## Récupération des données 

Les 3 blocs de code suivants permettent de récupérer les données  de la variable choisie de puis le serveur en ligne du projet. Les impressions sont facultatives, elles permettent de vérifier le bon déroulement de l'pération.

In [3]:
tMax_26_Lyon = nc.Dataset('http://geoloc-tremplin.ens-lyon.fr/climato-data/Lyon-1/tasmax_metro_CNRM_Aladin_rcp2.6_QT_RCP2.6_20060101-21001231.nc',casting='safe')
lyon_temp_26 = tMax_26_Lyon.variables['tasmax']
print('Structure et taille du tableau exporté :\n\t',tMax_26_Lyon.variables['tasmax'].dimensions, lyon_temp_26.shape)
print('\nPremière ligne de données :\n',lyon_temp_26[0,0,:])
print('Dernière ligne de données :\n',lyon_temp_26[-1,-1,:])

Structure et taille du tableau exporté :
	 ('time', 'j', 'i') (34698, 10, 10)

Première ligne de données :
 [279.86902 279.28592 278.47507 278.4666  283.58368 283.09763 282.4597
 281.01205 280.87045 280.77097]
Dernière ligne de données :
 [280.27753 280.44775 279.89688 280.8853  280.8976  280.38654 280.29166
 280.75772 281.7561  281.78995]


In [4]:
tMax_45_Lyon = nc.Dataset('http://geoloc-tremplin.ens-lyon.fr/climato-data/Lyon-1/tasmax_metro_CNRM_Aladin_rcp4.5_QT_RCP4.5_20060101-21001231.nc',casting='safe')
lyon_temp_45 = tMax_45_Lyon.variables['tasmax']
print('Structure et taille du tableau exporté :\n\t',tMax_45_Lyon.variables['tasmax'].dimensions, lyon_temp_45.shape)
print('\nPremière ligne de données :\n',lyon_temp_45[0,0,:])
print('Dernière ligne de données :\n',lyon_temp_45[-1,-1,:])

Structure et taille du tableau exporté :
	 ('time', 'j', 'i') (34698, 10, 10)

Première ligne de données :
 [279.8828  279.30203 278.5122  278.50308 283.59985 283.10727 282.46997
 281.0012  280.8424  280.73636]
Dernière ligne de données :
 [281.43195 281.6765  282.77155 283.65445 283.545   281.96106 281.90695
 282.3796  281.89853 281.91562]


In [5]:
tMax_85_Lyon = nc.Dataset('http://geoloc-tremplin.ens-lyon.fr/climato-data/Lyon-1/tasmax_metro_CNRM_Aladin_rcp8.5_QT_RCP8.5_20060101-21001231.nc',casting='safe')
lyon_temp_85 = tMax_85_Lyon.variables['tasmax']          
print('Structure et taille du tableau exporté :\n\t',tMax_85_Lyon.variables['tasmax'].dimensions, lyon_temp_85.shape)
print('\nPremière ligne de données :\n',lyon_temp_85[0,0,:])
print('Dernière ligne de données :\n',lyon_temp_85[-1,-1,:])

Structure et taille du tableau exporté :
	 ('time', 'j', 'i') (34698, 10, 10)

Première ligne de données :
 [279.8736  279.29126 278.55438 278.5493  283.59955 283.07462 282.4395
 281.00513 280.87564 280.77737]
Dernière ligne de données :
 [281.74222 281.9948  280.26053 281.42868 281.78946 282.9065  282.8423
 283.06964 282.7308  282.75534]


## Récupération des paramètres de l'extraction (dates, i-j Aladin, lat-lon, x-y) 

À partir de la liste des variables obtenue ci-dessus on renomme les jeux de données de chacune des variables qui seront exploitées apour effectuer les calculs et contrôle de la taille des échantillons. Les affichages proposés permettent de contrôler que les paramètres présents sont effectivement ceux qui sont attendus.

In [6]:
# On utilise ici le calcul 2.6 mais les dates sont identiques sur les 3 jeux de données
# une seule extraction est nécessaire
lyon_date = tMax_26_Lyon.variables['time']  
print('Taille du tableau de dates - ',lyon_date.shape)
print('Date de début de la simulation : ',nc.num2date(lyon_date[0],lyon_date.units).strftime("%c"))
print('Date de fin de la simulation : ',nc.num2date(lyon_date[-1],lyon_date.units).strftime("%c"))

Taille du tableau de dates -  (34698,)
Date de début de la simulation :  Sun Jan  1 00:00:00 2006
Date de fin de la simulation :  Fri Dec 31 00:00:00 2100


In [7]:
lyon_lat,lyon_lon = tMax_26_Lyon.variables['lat'], tMax_26_Lyon.variables['lon'] # latitude longitude
print('Emprise du projet en latititude-Longitude ;\n', lyon_lat[0][0],'#',lyon_lon[0][0],' :: ',lyon_lat[-1][-1],'#',lyon_lon[-1][-1])
lyon_x,lyon_y = tMax_26_Lyon.variables['x'], tMax_26_Lyon.variables['y']  # coordonnées métriques
print('Emprise du projet en x-y mètres ;\n', lyon_x[0],'#',lyon_y[0],' :: ',lyon_x[-1],'#',lyon_y[-1])
lyon_gridi,lyon_gridj = tMax_26_Lyon.variables['i'], tMax_26_Lyon.variables['j'] # coordonnées grille Aladin
print('Emprise du projet en noeuds ALADIN ;\n', lyon_gridi[0],'#',lyon_gridj[0],' :: ',lyon_gridi[-1],'#',lyon_gridj[-1])

Emprise du projet en latititude-Longitude ;
 45.423668 # 4.3302917  ::  46.050774 # 5.2839203
Emprise du projet en x-y mètres ;
 756000 # 2049000  ::  828000 # 2121000
Emprise du projet en noeuds ALADIN ;
 88.0 # 55.0  ::  97.0 # 64.0


## Création d'un fichier de sauvegarde des moyennes mensuelles

Il a été pris le parti d'aborder les données sous la forme de moyennes mensuelles entre 2006 et 2100, ces données pourront ensuite être exploitées pour afficher l'évolution des tendances climatiques sur des intervalles de temps choisis par l'utilisateur. Ce chapitre inclut les deux étapes de création du container puis de calcul des valeurs des moyennes.

Le format choisi est le même que celui utilisé par météofrance pour fournir les données [NETCDF](https://www.unidata.ucar.edu/software/netcdf/), ce format est complexe mais permet aisément de stocker des données multidimensionnelles et il est accompagné d'outils numériques permettant d'explorer efficacement cet espace multidimensionnel.

Le premier bloc **effece un fichier qui porterait le même nom** et crée un nouveau fichier vide pour accueillir les valeurs moyennes calculées pour chaque noeud de la grille  traitée.


In [8]:
try: os.remove('tsmaxLyon-26-45-85.nc')  # par sécurité efface le fichier portadatain = numpy.array(['foo','bar'],dtype='S3')nt ce nom ! attention aux pertes possibles
except OSError : pass
extractLyonTempYearMonth = nc.Dataset('tsmaxLyon-26-45-85.nc',mode='w',format='NETCDF4') 

## Construction du jeu de paramètres : année mois latitude longitude x y

Définition et affectation des variables où sont copiées les paramètres de la grille et où seront stockés les résultats des calculs. Les affichages permettent de vérifier la validité des données utilisées.


In [9]:
# tableau du nom des mois
listmonth = np.array(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul',
            'Aug', 'Sep', 'Oct', 'Nov', 'Dec', 'All'])
# Tableau du nombre de jours par mois année ordinaire
lenMonthA =[31,28,31,30,31,30,31,31,30,31,30,31]
# Tableau du nombre de jours par mois année bissextile
lenMonthB =[31,29,31,30,31,30,31,31,30,31,30,31]
# Conversion en nombre entier des années extrêmes
firstyear = int(nc.num2date(lyon_date[0],lyon_date.units).strftime("%Y"))
lastyear = int(nc.num2date(lyon_date[-1],lyon_date.units).strftime("%Y"))
# Détermination de la taille des tableaux de calcul
sizemonths = len(listmonth)
sizeyears = lastyear - firstyear + 1 
sizegridi = lyon_gridi.shape[0]
sizegridj = lyon_gridj.shape[0]
print ("DateTime de départ  de l'étude : ", firstyear,
      "\nDateTime de fin  de l'étude : ", lastyear,
      "\nDurée de l'étude : ", sizeyears, 'ans')
#
# Création des dimensions du tableau mois et année sont séparés pour disjoindre les traitements sur ces variables
#
extractLyonTempYearMonth.createDimension('i', sizegridi)     # latitude axis
extractLyonTempYearMonth.createDimension('j', sizegridj)    # longitude axis
extractLyonTempYearMonth.createDimension('month', sizemonths)    # month axis
extractLyonTempYearMonth.createDimension('year', sizeyears) # year axis
# Define two variables with the same names as dimensions,
# a conventional way to define "coordinate variables".
i = extractLyonTempYearMonth.createVariable('i', 'i4', ('i',))
i.long_name = 'cell index along first dimension'
j = extractLyonTempYearMonth.createVariable('j', 'i4', ('j',))
j.long_name = 'cell index along second dimension'
lat = extractLyonTempYearMonth.createVariable('lat', 'f4', ('j','i'))
lat.units = 'degrees_north'
lat.long_name = 'latitude'
lat.standard_name = 'latitude'
lat._CoordinateAxisType = 'Lat'
lon = extractLyonTempYearMonth.createVariable('lon', 'f4', ('j','i',))
lon.units = 'degrees_east'
lon.long_name = 'longitude'
lon.standard_name = 'longitude'
lon._CoordinateAxisType = 'Lon'
x = extractLyonTempYearMonth.createVariable('x', 'i4', ('i',))
x.long_name = 'cell index along first dimension'
y = extractLyonTempYearMonth.createVariable('y', 'i4', ('j',))
y.long_name = 'cell index along second dimension'
month = extractLyonTempYearMonth.createVariable('month','S3', ('month',))
month.units = 'month'
month.long_name = 'month_name'
month.standard_name = 'month_name'
year = extractLyonTempYearMonth.createVariable('year', 'u4', ('year',))
year.units = 'date'
year.long_name = 'year'
year.standard_name = 'year'
i[:] = lyon_gridi[:]
j[:] = lyon_gridj[:]
lat[:] = lyon_lat[:,:]
lon[:] = lyon_lon[:,:]
x[:] = lyon_x[:]
y[:] = lyon_y[:]
month[:] = listmonth

temp_26 = extractLyonTempYearMonth.createVariable('temp_26','f4',('year','month','j','i'),fill_value=1.e+20)
temp_26.units = 'degree C' # degrees Celsius
temp_26.standard_name = 'air_temperature_26' # this is a CF standard name
temp_45 = extractLyonTempYearMonth.createVariable('temp_45','f4',('year','month','j','i'),fill_value=1.e+20)
temp_45.units = 'degree C' # degrees Celsius
temp_45.standard_name = 'air_temperature_45' # this is a CF standard name
temp_85 = extractLyonTempYearMonth.createVariable('temp_85','f4',('year','month','j','i'),fill_value=1.e+20)
temp_85.units = 'degree C' # degrees Celsius
temp_85.standard_name = 'air_temperature-85' # this is a CF standard name
extractLyonTempYearMonth.title = 'Extrait TSMax par moyenne mensuelle de 2006 a 2100 Lyon et sa region'
extractLyonTempYearMonth.institution = 'ENS de Lyon'
extractLyonTempYearMonth.institute_id = 'IFE Institut Francais de l Education'
extractLyonTempYearMonth.project_id = 'Climat et meteo tremplin pour l enseignement des sciences'
extractLyonTempYearMonth.model_id = 'CNRM-ALADIN52'
extractLyonTempYearMonth.product = 'output derived from Meteofrance DRIAS data'
extractLyonTempYearMonth.contact = 'gerard.vidal@ens-lyon.fr'
extractLyonTempYearMonth.creation_date = str(datetime.datetime.now())
extractLyonTempYearMonth.driving_experiment_name = 'DRIAS2014'
extractLyonTempYearMonth.experiment = 'RCP2.6 RCP4.5 RCP8.5 '
extractLyonTempYearMonth.model = 'ALADIN-Climat'
extractLyonTempYearMonth.author = 'Gerard Vidal'
extractLyonTempYearMonth.comment = "Extraction des moyennes de la region Lyonnaise de 2006 a 2100 et changegement des variables"

DateTime de départ  de l'étude :  2006 
DateTime de fin  de l'étude :  2100 
Durée de l'étude :  95 ans


In [10]:
#print('température moyenne année 2015 noeud 5 5 : \n', temp[9,:,5,5])
#print("température moyenne mois d'avril sur les 95 années noeud 8 4 : \n", temp[:,3,8,4])
#print("température moyenne année 2050 mois d'aout tous les  noeuds : \n", temp[45,7,:,:])
#print('i :\n', i,i[:])
#print('j :\n', j,j[:])
#print('lon :\n', lon,lon[:,:])
#print('lat :\n', lat,lat[:,:])
#print('x :\n', x,x[:])
#print('y :\n', y,y[:])
#print('month :\n', month, '\n',datain,datain[:])
#print('i :\n',   extractLyonTempYearMonth.variables['i'][:])
#print('x :\n',   extractLyonTempYearMonth.variables['x'][:])
#print('lat :\n', extractLyonTempYearMonth.variables['lat'][:])
#print('month :', extractLyonTempYearMonth.variables['month'][:])

## Calcul principal des moyennes par mois pour chaque noeud et toutes les années


In [11]:
iteri = 0
iterj = 0
iteriFirst = iteri  
while iteri  < lyon_temp_26.shape[0] :      
    for iterj in  range(sizeyears) :
        year[iterj] = (nc.num2date(lyon_date[iteri],lyon_date.units).strftime("%Y"))
#        print(nc.num2date(lyon_date[iteri],lyon_date.units), iteri, iterj)  # vérification du point de départ
        print(iterj,year[iterj] )
        if (iterj % 4 == 2) :
            for p in range(len(lenMonthB)) :
                iteriLast = iteri + lenMonthB[p]
                # moyenne du mois année bissextile
                temp_26[iterj,p,:,:] = np.mean(lyon_temp_26[iteri:iteriLast,:,:] - 273,axis=0) 
                temp_45[iterj,p,:,:] = np.mean(lyon_temp_45[iteri:iteriLast,:,:] - 273,axis=0) 
                temp_85[iterj,p,:,:] = np.mean(lyon_temp_85[iteri:iteriLast,:,:] - 273,axis=0) 
                iteri = iteriLast
        else :
            for p in range(len(lenMonthA)) :
                iteriLast = iteri + lenMonthA[p]
                # moyenne du mois année ordinaire
                temp_26[iterj,p,:,:] = np.mean(lyon_temp_26[iteri:iteriLast,:,:] - 273,axis=0) 
                temp_45[iterj,p,:,:] = np.mean(lyon_temp_45[iteri:iteriLast,:,:] - 273,axis=0) 
                temp_85[iterj,p,:,:] = np.mean(lyon_temp_85[iteri:iteriLast,:,:] - 273,axis=0) 
                iteri = iteriLast
       # moyenne de l'année
        temp_26[iterj,len(lenMonthA),:,:] = np.mean(lyon_temp_26[iteriFirst:iteriLast,:,:] - 273,axis=0) 
        temp_45[iterj,len(lenMonthA),:,:] = np.mean(lyon_temp_45[iteriFirst:iteriLast,:,:] - 273,axis=0) 
        temp_85[iterj,len(lenMonthA),:,:] = np.mean(lyon_temp_85[iteriFirst:iteriLast,:,:] - 273,axis=0) 
        iteriFirst = iteriLast+1
#        print('Températures moyennes annuelles des noeuds extrêmes de la grille : ',
#              temp_26[iterj,len(lenMonthA),0,0],temp_26[iterj,len(lenMonthA),-1,-1],
#              temp_45[iterj,len(lenMonthA),0,0],temp_45[iterj,len(lenMonthA),-1,-1],
#              temp_85[iterj,len(lenMonthA),0,0],temp_85[iterj,len(lenMonthA),-1,-1])
# extraction d'une moyenne au hasard (aout 2050)
print('\n Valeur moyenne 3 scénarios Lyon aout 2050 :\n',temp_26[44,7,5,5],temp_45[44,7,5,5],temp_85[44,7,5,5])

0 2006
1 2007
2 2008
3 2009
4 2010
5 2011
6 2012
7 2013
8 2014
9 2015
10 2016
11 2017
12 2018
13 2019
14 2020
15 2021
16 2022
17 2023
18 2024
19 2025
20 2026
21 2027
22 2028
23 2029
24 2030
25 2031
26 2032
27 2033
28 2034
29 2035
30 2036
31 2037
32 2038
33 2039
34 2040
35 2041
36 2042
37 2043
38 2044
39 2045
40 2046
41 2047
42 2048
43 2049
44 2050
45 2051
46 2052
47 2053
48 2054
49 2055
50 2056
51 2057
52 2058
53 2059
54 2060
55 2061
56 2062
57 2063
58 2064
59 2065
60 2066
61 2067
62 2068
63 2069
64 2070
65 2071
66 2072
67 2073
68 2074
69 2075
70 2076
71 2077
72 2078
73 2079
74 2080
75 2081
76 2082
77 2083
78 2084
79 2085
80 2086
81 2087
82 2088
83 2089
84 2090
85 2091
86 2092
87 2093
88 2094
89 2095
90 2096
91 2097
92 2098
93 2099
94 2100

 Valeur moyenne 3 scénarios Lyon aout 2050 :
 27.21591 27.036688 31.908592


Les affichages suivants permettent de vérifier  que les données obtenues correspondent au format attendu et présentent des valeurs cohérentes, plusieurs types de représentation sont proposés.

In [12]:
#print('température moyenne année 2015 noeud 5 5 : \n', temp[9,:,5,5])
#print("température moyenne mois d'avril sur les 95 années noeud 8 4 : \n", temp[:,3,8,4])
#print("température moyenne année 2050 mois d'aout tous les  noeuds : \n", temp[45,7,:,:])
#print('i :\n', i,i[:])
#print('lon :\n', lon,lon[:,:])
#print('x :\n', x,x[:])
#print('year :\n', year, year[:])
#print('month :', month, month[:])
#print(extractLyonTempYearMonth.variables['i'][:])
#print(extractLyonTempYearMonth.variables['month'][:])

In [13]:
extractLyonTempYearMonth.close()